In [10]:
# 1) 필요한 라이브러리 임포트
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection
)
import polars as pl
import os

# 2) Milvus 서버 연결
connections.connect(alias="default", host="localhost", port="19530")

# 3) Parquet 파일 로드 (경로 수정)
df = pl.read_parquet("/Users/gim-yonghyeon/Documents/GitHub/image-search/dataset_embeddings.parquet")
print("Columns in DataFrame:", df.columns)

# 4) 컬럼명에 맞춰 값 추출
raw_paths  = df["image_path"].to_list()
filenames  = [os.path.basename(p) for p in raw_paths]
embeddings = df["embedding"].to_list()
dim = len(embeddings[0])

# 5) 기존 'images' 컬렉션 삭제(있으면)
if utility.has_collection("images"):
    utility.drop_collection("images")

# 6) 컬렉션 스키마 정의 (auto ID + filename + embedding)
fields = [
    FieldSchema(name="id",       dtype=DataType.INT64,      is_primary=True, auto_id=True),
    FieldSchema(name="filename", dtype=DataType.VARCHAR,    is_primary=False, max_length=256),
    FieldSchema(name="embedding",dtype=DataType.FLOAT_VECTOR, dim=dim)
]
schema = CollectionSchema(fields, description="Image embeddings with auto ID")

# 7) 컬렉션 생성
images_col = Collection(name="images", schema=schema)

# 8) 데이터 삽입
images_col.insert([filenames, embeddings])

# 9) Flush 후 개수 확인
images_col.flush()
print(f"✅ Total entities in 'images' collection: {images_col.num_entities}")

# 10) 인덱스 생성 및 로드
images_col.create_index(
    field_name="embedding",
    index_params={
        "index_type": "IVF_FLAT",
        "metric_type": "COSINE",
        "params": {"nlist": 128}
    }
)
images_col.load()
print(f"'images' collection is loaded (dim={dim}). 검색 준비 완료.")

Columns in DataFrame: ['image_path', 'embedding']
✅ Total entities in 'images' collection: 12813
'images' collection is loaded (dim=1024). 검색 준비 완료.


In [4]:
# 1) polars로 Parquet 읽고 컬럼명·샘플 출력
import polars as pl

df = pl.read_parquet("/Users/gim-yonghyeon/Documents/GitHub/image-search/dataset_embeddings.parquet")
print("Columns:", df.columns)
print(df.head())

Columns: ['image_path', 'embedding']
shape: (5, 2)
┌─────────────────────────────────┬─────────────────────────────────┐
│ image_path                      ┆ embedding                       │
│ ---                             ┆ ---                             │
│ str                             ┆ list[f64]                       │
╞═════════════════════════════════╪═════════════════════════════════╡
│ 08012003/08_082_08012003_16098… ┆ [0.014125, 0.004899, … -0.0012… │
│ 08012003/08_082_08012003_16099… ┆ [0.024131, 0.016818, … -0.0119… │
│ 08012003/08_082_08012003_16095… ┆ [-0.011053, 0.036774, … -0.054… │
│ 08012003/08_082_08012003_16099… ┆ [0.01507, -0.01684, … 0.001349… │
│ 08012003/08_082_08012003_16099… ┆ [-0.000712, -0.001423, … -0.02… │
└─────────────────────────────────┴─────────────────────────────────┘


In [ ]:
# Milvus 컬렉션 조회 및 Parquet 파일 저장 예시 (Jupyter Notebook)

from pymilvus import (
    connections,
    utility,
    Collection,
)
import numpy as np
import pandas as pd

# 1) Milvus 서버 연결
connections.connect(
    alias="default",
    host="localhost",
    port="19530"
)

# 2) 모든 컬렉션 이름 리스트 확인
print("=== Milvus에 존재하는 컬렉션 목록 ===")
all_collections = utility.list_collections()
for idx, name in enumerate(all_collections, start=1):
    print(f"{idx}. {name}")

# 3) 조회할 컬렉션 이름 지정
collection_name = input("저장할 컬렉션 이름을 입력하세요: ")
if collection_name not in all_collections:
    raise ValueError(f"컬렉션 '{collection_name}' 이 존재하지 않습니다.")

# 4) 컬렉션 로드
col = Collection(collection_name)

# 5) 전체 엔티티 조회
print(f"컬렉션 '{collection_name}' 데이터 조회 중...")
data = col.query(expr="id >= 0", output_fields=[f.name for f in col.schema.fields])

# 6) DataFrame으로 변환
# 리스트 of dict -> pandas DataFrame
records = []
for record in data:
    row = {}
    for key, value in record.items():
        if isinstance(value, (np.generic,)):
            row[key] = value.item()
        elif isinstance(value, np.ndarray):
            row[key] = value.tolist()
        else:
            row[key] = value
    records.append(row)

df = pd.DataFrame(records)

# 7) Parquet 파일로 저장
output_path = f"{collection_name}_export.parquet"
df.to_parquet(output_path, index=False)
print(f"데이터 저장 완료: {output_path}")

# 8) 저장된 Parquet 데이터 확인
print("=== 저장된 Parquet 파일 데이터 미리 보기 ===")
# 파일에서 다시 읽어와서 확인
df_loaded = pd.read_parquet(output_path)
print(df_loaded.head())

In [ ]:
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm

import torch
from transformers import AutoProcessor, AutoModel
import polars as pl

# 1) 모델 로드
model_name = "google/siglip-large-patch16-384"
processor = AutoProcessor.from_pretrained(
    model_name,
    trust_remote_code=True,
    use_fast=True
)
model = AutoModel.from_pretrained(
    model_name,
    trust_remote_code=True
).eval()

# 2) MPS 디바이스 설정 (Mac M1/M2/M3)
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("▶ Using Apple MPS")
else:
    device = torch.device("cpu")
    print("▶ Using CPU")

model.to(device)

# 3) 이미지 파일 경로 수집
image_root = Path("/Users/gim-yonghyeon/Documents/GitHub/image-search/images")
image_paths = list(image_root.rglob("*.[jJ][pP][gG]")) + \
              list(image_root.rglob("*.[jJ][pP][eE][gG]")) + \
              list(image_root.rglob("*.[pP][nN][gG]"))

# 4) 임베딩 추출
records = []
for img_path in tqdm(image_paths, desc="Embedding images"):
    try:
        img = Image.open(img_path).convert("RGB")
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            feats = model.get_image_features(**inputs)
            feats = feats / feats.norm(dim=-1, keepdim=True)
        vec = feats.squeeze(0).cpu().tolist()
        records.append({
            "image_path": str(img_path),
            "embedding": vec
        })
    except Exception as e:
        print(f"⚠️ 실패: {img_path} → {e}")

# 5) Polars DataFrame 생성 및 Parquet 저장
df = pl.DataFrame(records)
output_file = "all_images_embeddings.parquet"
df.write_parquet(output_file)
print(f"✅ 저장 완료: {output_file} ({df.height}개 레코드)")

▶ Using Apple MPS


Embedding images:   0%|                    | 64/12813 [00:17<1:00:14,  3.53it/s]